In [51]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

# change this as you see fit
image_path = "geometry_data/angle/ee540fbd068c495da4e52af8d7834b3c.jpg"

# Read in the image_data
image_data = tf.gfile.FastGFile(image_path, 'rb').read()

img1 = mpimg.imread(image_path)
test_img = cv2.resize(img1[0:100, 0:100], (299,299))
np_image_data = np.asarray(test_img)
np_image_data=cv2.normalize(np_image_data.astype('float'), None, -0.5, .5, cv2.NORM_MINMAX)
#maybe insert float convertion here - see edit remark!
image_data = test_img = np.expand_dims(np_image_data,axis=0)

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("retrained_labels.txt")]

# Unpersists graph from file
with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

with tf.Session() as sess:
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
    #predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})    
    predictions = sess.run(softmax_tensor, {'Mul:0': image_data})
    
    # Sort to show labels of first prediction in order of confidence
    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
    
    for node_id in top_k:
        human_string = label_lines[node_id]
        score = predictions[0][node_id]
        print('%s (score = %.5f)' % (human_string, score))

angle (score = 0.99487)
arrow (score = 0.00427)
line (score = 0.00085)


In [48]:
import os
import os.path
from shutil import copyfile, rmtree
import uuid
from PIL import Image

def prepare_data(folder_from, folder_to):
    print("start")
    
    # clear folder
    rmtree(folder_to)
    
    files = []
    for dirpath, dirnames, filenames in os.walk(folder_from):
        for filename in [f for f in filenames if (f.endswith(".jpg") or f.endswith(".png"))]:
            # get path to file
            file_path = os.path.join(dirpath, filename)
            files.append(file_path)
            
            # get class
            start = file_path.index("/", 0) + 1
            end = file_path.index("/", start)
            class_name = file_path[start:end]
            
            # create class folder 
            if not os.path.exists(folder_to + "/" + class_name):
                os.makedirs(folder_to + "/" + class_name)
            
            # copy file
            file_name = uuid.uuid4().hex + ".jpg"
            
            if(filename.endswith(".jpg")):
                copyfile(file_path, folder_to + "/" + class_name + "/" + file_name)
            elif (filename.endswith(".png")):
                Image.open(file_path).save(folder_to + "/" + class_name + "/" + file_name)            
    
    print("processed files: ", len(files))

In [49]:
#prepare_data("data_tree", "geometry_data")

start
('processed files: ', 10)
